<a href="https://colab.research.google.com/github/CrzPhil/IN3063-Coursework/blob/main/Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IN3063 - Coursework

## Libraries

In [ ]:
import math
import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt

## Sigmoid & ReLU

- By Aymen
- Reference:
    - https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795
    - https://www.sharpsightlabs.com/blog/numpy-relu/
    - Lab 6

In [ ]:
# Forward pass for Sigmoid
def forward_sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Backward pass for Sigmoid
def backward_sigmoid(x):
    return forward_sigmoid(x) * (1 - forward_sigmoid(x))

In [ ]:
# Forward pass for ReLU
def forward_relu(x):
    return np.maximum(0, x)

# Backward pass for ReLU
def backward_relu(x):
    return np.where(x > 0, 1, 0)

## Softmax

- By Aymen
- Using the Numpy version
- Reference:
    - https://towardsdatascience.com/softmax-function-simplified-714068bf8156
    - https://en.wikipedia.org/wiki/Softmax_function
    - https://www.sharpsightlabs.com/blog/numpy-softmax/

In [ ]:
# Forward pass for Softmax
def forward_softmax(x):
    exponential = np.exp(x - np.max(x))
    return exponential / exponential.sum() # calculates softmax probability

# Backward pass for Softmax
def backward_softmax(x):
    return np.reshape(forward_softmax(x) * (1 - forward_softmax(x)), (1, -1)) # computes gradient of softmax

# Testing:
x = np.array([100.0, 2000.0, 300.0]) # large numbers
print("Forward pass result:", forward_softmax(x))
print("Backward pass result:", backward_softmax(x))
print ("\n")

x = np.array([1.0, 2.0, 3.0]) # small numbers
print("Forward pass result:", forward_softmax(x))
print("Backward pass result:", backward_softmax(x))

Forward pass result: [0. 1. 0.]
Backward pass result: [[0. 0. 0.]]


Forward pass result: [0.09003057 0.24472847 0.66524096]
Backward pass result: [[0.08192507 0.18483645 0.22269543]]


## Dropout

- By Adam
- References
  - Lecture 7
  - https://stackoverflow.com/questions/70836518/typeerror-bad-operand-type-for-unary-list-python

In [ ]:
def dropout(x, probability, activation_function, train): # x = input vector, probability is a float, activation_function is a string, train is a boolean
    activation_functions = {
        "sigmoid": forward_sigmoid,
        "relu": forward_relu,
        "softmax": forward_softmax
    }

    try:
        H1 = activation_functions[activation_function](x)
    except KeyError:
        print("Invalid activation function passed")
        return -1

    if train:
        mask = (np.random.rand(*H1.shape) < probability) / probability
        return H1 * mask
    else:
        return H1

# Testing: Training
x = np.arange(2.0, 4.0, 7.0)
H1_dropped = dropout(x, 0.5, "sigmoid", True)
print(H1_dropped)

# Testing: Testing
H1_dropped2 = dropout(x, 0.5, "sigmoid", False)
print(H1_dropped2)

[1.76159416]
[0.88079708]


## Neural Network
- By Philip  
Implement a fully parametrizable neural network class
You should implement a fully-connected NN class where with number of
hidden layers, units, activation functions can be changed. In addition, you
can add dropout or regularizer (L1 or L2). Report the parameters used
(update rule, learning rate, decay, epochs, batch size) and include the plots
in your report.

In [ ]:
# Reading the MNIST dataset as per http://yann.lecun.com/exdb/mnist/
import os
import struct

def read_idx(filename):
    with open(filename, 'rb') as file:
        # Read two bytes (big endian and unsigned)
        zero, data_type, dims = struct.unpack('>HBB', file.read(4))
        # Four byte integer big endian
        shape = tuple(struct.unpack('>I', file.read(4))[0] for d in range(dims))
        return np.frombuffer(file.read(), dtype=np.uint8).reshape(shape)

def load_mnist(path):
    # Paths to the files
    train_images_path = os.path.join(path, 'train-images-idx3-ubyte')
    train_labels_path = os.path.join(path, 'train-labels-idx1-ubyte')
    test_images_path = os.path.join(path, 't10k-images-idx3-ubyte')
    test_labels_path = os.path.join(path, 't10k-labels-idx1-ubyte')

    # Loading the datasets
    train_images = read_idx(train_images_path)
    train_labels = read_idx(train_labels_path)
    test_images = read_idx(test_images_path)
    test_labels = read_idx(test_labels_path)

    return train_images, train_labels, test_images, test_labels

In [ ]:
# Example use
t_images, t_labels, test_images, test_labels = load_mnist('./dataset')

NameError: ignored

In [ ]:
class NeuralNet:
  def __init__(self, activation_function, layers, batch_size, neurons):
    """
    Initialises a new instance of the NeuralNet class.

    Parameters:
    activation_function (func): The activation function to be used in the network layers.
                                The function is used in all layers.
    layers (int): The number of layers in the neural network.
    batch_size (int): The size of the batches used in training. This affects how the data is split during training iterations.
    neurons (list of int): The number of neurons in each layer. This should be a list where each element represents
                            the number of neurons in the respective layer of the network.
    Returns:
    None
    """
    self.activation_function = activation_function
    self.layers = layers
    self.batch_size = batch_size
    self.neurons = neurons
    # Will be initialised once features are known
    self.weights = []
    self.biases = []

  def init_weights_and_biases(self, input_features):
    # Initialise weights and biases based on the layers, neurons, and input features
    # Fully connected through weights
    for i in range(self.layers):
      if i == 0:
          layer_weights = np.random.randn(self.neurons[i], input_features) * 0.01
      else:
          layer_weights = np.random.randn(self.neurons[i], self.neurons[i - 1]) * 0.01
      layer_bias = np.zeros((self.neurons[i], 1))
      self.weights.append(layer_weights)
      self.biases.append(layer_bias)

  def forward_pass(self, X):
    activations = [X]
    for i in range(self.layers):
      Z = np.dot(self.weights[i], activations[-1]) + self.biases[i]
      A = self.apply_activation(Z)
      activations.append(A)
    return activations

  def apply_activation(self, Z):
    return self.activation_function(Z)

  def backward_pass(self, X, Y):
    pass

  def update_weights_and_biases(self, gradients, learning_rate):
    for i in range(self.layers):
      pass

  def train_network(self, epochs, batch_size, learning_rate, X_train, Y_train):
    for epoch in range(epochs):
      # Maybe shuffle training data before batching it?

      # Iterate batches
      for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i:i + batch_size]
        Y_batch = Y_train[i:i + batch_size]

        # Forward pass over the batch
        activations = self.forward_pass(X_batch)

        # Backward pass over the batch (get gradients)
        gradients = self.backward_pass(activations, Y_batch)

        # Update weights & biases
        self.update_weights_and_bies(gradients, learning_rate)


  def evaluate_model(self, x_test, y_test, X_train, Y_train, loss_list):
    pass